## 最后要把所有的tqdm删掉

In [ ]:
# 类外函数名为addFun格式
# 类内函数名及变量为add_fun格式
# 类格式AddFun

In [32]:
import os
import re
import sys
import json

from tqdm import tqdm
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import Levenshtein
from fuzzywuzzy import fuzz
import redis
r = redis.Redis('localhost', 6379)

# 中文标题的hashcode不能全置0，要改
from tool import hashString,hashStringOdd,hashStringEven,hashString3,containChinese
from H5File import H5File
from generate_id import IdWorker
worker = IdWorker(1,2)
from merge import mergeColumnAndTitle

max_bucket_size = 0
repeat_count = 0
repeat_2 = 0
result_list = []

columns_list = {}
with open('column_config.json','r') as f:
    columns_list = json.loads(f.read())

In [ ]:
"""
def main():
    h5file = H5File(3, 2021)
    h5file.generate_DataFrame()
    h5file.add_title_code()
    pdata = h5file.pdata


if __name__ == '__main__':
    main()
"""    

In [81]:
%%time
h5file = H5File(8, 2021)
h5file.generate_DataFrame()
# h5file.add_title_alter()
# h5file.add_id_mark()
h5file.add_title_code()

CPU times: user 274 ms, sys: 14.1 ms, total: 288 ms
Wall time: 286 ms


In [82]:
len(h5file.pdata)

14168

In [83]:
h5file.pdata.head()

,id,title,language,provider_subject,source_file,title_code
0,20855735608,广东省财政厅关于做好2020年度国有企业财务会计决算报告工作的通知,ZH,地方法规规章,111,广东省财政厅关于做好2020年度国有企业财务会计决算报告工作的通知
1,20855737066,郑州市商务局、郑州市财政局关于做好河南省跨境电子商务海外仓示范企业申报的通知,ZH,地方法规规章,111,郑州市商务局郑州市财政局关于做好河南省跨境电子商务海外仓示范企业申报的通知
2,20855748866,第十三届全国人民代表大会宪法和法律委员会关于《全国人民代表大会关于修改＜中华人民共和国全国人...,ZH,立法草案,111,第十三届全国人民代表大会宪法和法律委员会关于全国人民代表大会关于修改中华人民共和国全国人民代...
3,20855749144,生态环境部关于山西兰花科创玉溪煤矿有限责任公司玉溪煤矿新建项目变更环境影响报告书的批复,ZH,中央法规司法解释,111,生态环境部关于山西兰花科创玉溪煤矿有限责任公司玉溪煤矿新建项目变更环境影响报告书的批复
4,20855749595,2021年宁夏回族自治区政府工作报告,ZH,,111,2021年宁夏回族自治区政府工作报告


In [84]:
h5file.pdata.loc[[2519, 3672, 3709, 7970, 8140, 8506, 8881, 10648, 12846, 13903]]

,id,title,language,provider_subject,source_file,title_code
2519,20855805686,财政部办公厅关于2021年记账式附息(四期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,945,财政部办公厅关于2021年记账式附息四期国债发行工作有关事宜的通知
3672,20855806346,财政部办公厅关于2021年记账式附息(五期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,729,财政部办公厅关于2021年记账式附息五期国债发行工作有关事宜的通知
3709,20855846606,财政部办公厅关于2021年记账式附息(九期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,572,财政部办公厅关于2021年记账式附息九期国债发行工作有关事宜的通知
7970,20855847040,财政部办公厅关于2021年记账式附息(八期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,715,财政部办公厅关于2021年记账式附息八期国债发行工作有关事宜的通知
8140,20855805771,财政部办公厅关于2021年记账式附息(六期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,870,财政部办公厅关于2021年记账式附息六期国债发行工作有关事宜的通知
8506,20855735890,财政部办公厅关于2021年记账式附息(一期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,304,财政部办公厅关于2021年记账式附息一期国债发行工作有关事宜的通知
8881,20855807050,财政部办公厅关于2021年记账式附息(二期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,662,财政部办公厅关于2021年记账式附息二期国债发行工作有关事宜的通知
10648,20855847547,财政部办公厅关于2021年记账式附息(七期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,527,财政部办公厅关于2021年记账式附息七期国债发行工作有关事宜的通知
12846,20856179484,财政部办公厅关于2021年记账式附息(十期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,456,财政部办公厅关于2021年记账式附息十期国债发行工作有关事宜的通知
13903,20855748568,财政部办公厅关于2021年记账式附息(三期)国债发行工作有关事宜的通知,ZH,中央法规司法解释,230,财政部办公厅关于2021年记账式附息三期国债发行工作有关事宜的通知


In [ ]:
17795
String:	内蒙古自治区2002年政府工作报告
Index:	26493
String:	内蒙古自治区2004年政府工作报告
Index:	34497
String:	内蒙古自治区2002年政府工作报告
Index:	41982
String:	内蒙古自治区2003年政府工作报告
Index:	43137
String:	内蒙古自治区2004年政府工作报告

In [36]:
f = open('test8.txt', 'w', encoding='utf-8')
repeat_count = 0
hash_code_map_final_list = []
repeat_2 = 0

In [37]:
%%time

# 1. 先hash再辅助判断
if h5file.type_id in [1, 4, 8, 9, 12, 14]:
    hashTitleTri(h5file)
# 2. doi先去重再hash
elif type_id in [3, 5, 6, 7, 11]:
    column_dic, empty_pdata_index, distinct_pdata_index = dedupWithColumn(h5file)
    title_dic = hashTitleHex(h5file, empty_pdata_index, distinct_pdata_index)
    mergeColumnAndTitle(id_set, column_dic, title_dic)
    # r.hset(h5file.file_name + columns_list[str(type_id)][0], list(column_dic.keys())[0], json.dumps(column_dic)) 
# 直接hash:0,2,10,13
else:
    hashTitleTri(h5file)

hash_code_map: 100%|██████████| 4/4 [00:15<00:00,  3.89s/it]

Finish.
CPU times: user 16.7 s, sys: 382 ms, total: 17.1 s
Wall time: 17.9 s


In [31]:
r.connection_pool.disconnect()

In [ ]:
def setNewIdSave(final_dic):
    """
    将确定相同的数据的列表存为同一个new_id
    """
    for k, index_list in final_dic.items():
        new_id = worker.get_id()
        for i in index_list:
            r.rpush(new_id, i)

In [2]:
def dedupWithColumn(h5file):
    """
    修改self.pdata，删除column重复项，保留column为空项，供后面title_hash，此处未重置索引
    返回三个变量：
    column_dic：存储所有数据中column不为空的数据，格式为{'doi':[属于这个doi的所有id...]}
    empty_pdata_index：column为空项的布尔索引
    distinct_pdata_index：column为空的数据中，删除column重复项后剩下的数据的布尔索引
    """
    """
    # 以下是旧版本代码，将h5file.pdata复制了三份，内存翻倍
    column_empty_pdata = h5file.pdata[h5file.pdata[column] == '']
    column_not_empty_pdata = h5file.pdata[h5file.pdata[column] != '']
    # 取一个别名(浅拷贝)
    pdata = column_not_empty_pdata
    # 存储所有数据中column不为空的数据，格式为{'doi':[属于这个doi的所有id...]}
    column_dic = {value:df.id.tolist() for value,df in pdata.groupby(pdata[column]) if value != ''}
    # 存储所有数据中column不为空的数据的column，格式为set
    column_set = set(pdata[pdata[column] != ''][column].unique())
    # 存储所有数据中column重复的数据（不含空值），格式为{'doi':[重复id...]}
    dup_pdata = pdata[pdata.duplicated(subset=column,keep=False)]
    dup_dic = {value:df.id.tolist() for value,df in dup_pdata.groupby(dup_pdata[column])}
    # dup_dic = {value:df.id.tolist() for value,df in pdata.groupby(pdata[column]) if len(df) > 1}
    # 修改self.pdata，删除column重复项，保留column为空项，供后面title_hash，此处重置索引
    h5file.pdata = pd.concat([pdata.drop_duplicates(subset=column),column_empty_pdata],axis=0)
    """
    column = columns_list[str(h5file.type_id)][0]
    empty_pdata_index = (h5file.pdata[column] == '')
    # 去重，保留第一次出现的数据，反转是为了使不重复的index为True
    distinct_pdata_index = (h5file.pdata.duplicated(subset=column,keep='first') == False) 
    # 存储所有数据中column不为空的数据，格式为{'doi':[属于这个doi的所有id...]}
    # column_dic = {value:df.id.tolist() for value,df in pdata.groupby(pdata[column]) if value != ''}
    column_dic = {}
    for value,df in tqdm(h5file.pdata.groupby(h5file.pdata[column]),desc='groupby'):
        if value != '':
            column_dic[value] = df.id.tolist()
            break
    
    # 修改self.pdata，删除column重复项，保留column为空项，供后面title_hash，此处重置索引
    # 谨慎修改self.pdata，因为post_proccess中可以会需要索引
    # h5file.pdata = h5file.pdata[empty_pdata_index|distinct_pdata_index]
    
    return column_dic, empty_pdata_index, distinct_pdata_index

In [9]:
def hashTitleTri(h5file):
    hash_code_map = {}
    for index,row in h5file.pdata.iterrows():
        # 获得hashcode
        hash_code = hashString(row['title_code'])
        # 将（hashcode， index）加入字典
        if hash_code in hash_code_map:
            hash_code_map[hash_code].append(index)
        else:
            hash_code_map[hash_code] = [index]
            
    # f = open('test.txt', 'w', encoding='utf-8')
    # f = open('/Users/jas/Desktop/test.txt', 'w')

    hash_code_map_final_list = []

    for hash_code,index_list in tqdm(hash_code_map.items(),desc='hash_code_map'): 
        # if(len(index_list)>1):
        # print('Hash_code:\t{}'.format(hash_code))
        # 对奇数位再哈希，两次相同则判断为同一字符 
        reHashDicOdd = {}
        for index in index_list:
            # 1111_2222
            hash_code_odd = str(hash_code) + '_' + str(hashStringOdd(h5file.pdata['title_code'][index]))
            # hash_code_odd = hashStringOdd(pdata['title_code'][index])
            if hash_code_odd in reHashDicOdd:
                reHashDicOdd[hash_code_odd].append(index)
            else:
                reHashDicOdd[hash_code_odd] = [index]

            # postPoccess(f, reHashDicOdd, pdata)

            # """
            # 再采用间隔哈希(每三个)
            # 再hash后max_bucket_size由600变为400，速度变为一半，召回个数减少约200个，约1%
        for hash_code,index_list in reHashDicOdd.items(): 
            reHashDic3 = {}
            for index in index_list:
                # hash_code_3 = hashString3(pdata['title_code'][index])
                # 1111_2222_3333
                hash_code_3 = hash_code + '_' + str(hashString3(h5file.pdata['title_code'][index]))
                if hash_code_3 in reHashDic3:
                    reHashDic3[hash_code_3].append(index)
                else:
                    reHashDic3[hash_code_3] = [index]
                # 精筛，注意这个代码要放在粗筛的最后一层，否则前面的筛选失效
            postPoccess(h5file, reHashDic3)
#             global postPoccess_count
#             if postPoccess_count > 5:
#                 assert 0
#             else:
#                 postPoccess_count += 1
    f.close()    
    print("Finish.")

In [17]:
def postPoccess(h5file, hashDic):
    """
    hash + 辅助判
    断
    对最终的hash表处理，找出相同的，写入文件
    """ 
    # print("postPoccessing, length of hasdDic:{}".format(len(hashDic)))
    global max_bucket_size
    global repeat_count
    global hash_code_map_final_list
    global repeat_2
    
    for hash_code,index_list in hashDic.items(): 
        r.hset(h5file.file_name + 'title_hash', hash_code, json.dumps({hash_code: h5file.pdata.loc[index_list]['id'].to_list()}))
        length = len(index_list)
        # 不需要用length删选了，因为如果length==1,那么以下的语句执行1次就会结束
        # if length > 1:
        max_bucket_size = max(max_bucket_size, length)
        # 使用numpy加快处理速度(维度大于10000时提速显著)
        visited = np.zeros([length+1], dtype = np.int)
        hash_code_map_final = {}
        for i in range(length):
            """
            # visited[i]:
            # 注意：如果该title已经并入其他title，则跳过（每个title只被匹配一次）
            # 如果不跳过，可以增加召回率
            # pdata['id_mark'][index_list[i]].endswith('a')):
            # 使hash_code_map_final[i]的i标题不是通过alt复制出来的新行，三种情况：
            # 1. 原标题（i）与alt（j）相似，alt被append到hash_code_map_final[原标题index]中
            # 2. alt（i）与原标题（j）相似，被1中情况覆盖，跳过
            # 3. alt（i）与alt（j）相似，一般alt对应的原标题也会相似，也跳过
            """
            if visited[i] :
                continue
            # 初始化
            hash_code_map_final[i] = [index_list[i]]
            visited[i] = 1
            for j in range(i+1, length):
                if visited[j] != 1:
                    # 提高效率
                    temp_i = h5file.pdata['title_code'][index_list[i]]
                    temp_j = h5file.pdata['title_code'][index_list[j]]
                    if containChinese(temp_i) and len(temp_i) < 15:
                        if fuzz.ratio(temp_i, temp_j) > 95:
                            # 需要辅助判断
                            if h5file.type_id in [1, 4, 8, 9, 12, 14]:
                                column = columns_list[str(h5file.type_id)][0]
                                if h5file.pdata[column][index_list[i]] == h5file.pdata[column][index_list[j]]:
                                    hash_code_map_final[i].append(index_list[j])
                                    visited[j] = 1
                            else: # 不需要辅助判断
                                hash_code_map_final[i].append(index_list[j])
                                visited[j] = 1
                    else:
                        if fuzz.ratio(temp_i, temp_j) > 85:
                            # 需要辅助判断
                            if h5file.type_id in [1, 4, 8, 9, 12, 14]:
                                column = columns_list[str(h5file.type_id)][0]
                                if h5file.pdata[column][index_list[i]] == h5file.pdata[column][index_list[j]]:
                                    hash_code_map_final[i].append(index_list[j])
                                    visited[j] = 1
                            else: # 不需要辅助判断
                                hash_code_map_final[i].append(index_list[j])
                                visited[j] = 1
                    """
                    这样会出现这种情况：
                    有一条记录：title：中文1（index：2），alt：英文a（index：4），分别与另外两条记录相似
                    title：中文2（index：1） 相似于 中文1
                    title：英文b（index：3） 相似于 英文a
                    处理后他们被归入两个字典：
                    {1:[1, 2]}
                    {3:[3, 4]}
                    他们都属于相似的标题，需要在最后，使用并查集操作使他们归并到同一个集合
                    """
            """
            
            # redis for j循环结束
            # 如果length==1那么不会进入j循环，直接存redis
            new_id_for_key = worker.get_id()
            # 重复列表中的index索引id
            old_dup_id_list = h5file.pdata.loc[hash_code_map_final[i]]['id'].to_list()
            # print(new_id_for_key,':',old_dup_id_list)

            for old_id in old_dup_id_list:
                r.rpush(new_id_for_key, old_id)   # {new_id: [dup_old_id, , ]}
                r.set(old_id, new_id_for_key)   # {old_id : new_id}
            """

        hash_code_map_final_list.append(hash_code_map_final)
        for hash_code,index_list in hash_code_map_final.items():  # hash_code 为index
            if (len(index_list)) > 1:
                # redis

                repeat_count += (len(index_list)-1)
                if (len(index_list)) == 2:
                    repeat_2 += 1  
                f.writelines(['*********************************************Same ReHash*********************************************\n'])
                for index in index_list:
                    # repeat_count += 1
                    f.writelines(['Index:\t', str(index), '\n', 'String:\t', h5file.pdata['title'][index], '\n'])                  


In [ ]:
def del_symbol(s):
    # return s.upper().translate(punc_table)
    tmp_s = s.translate(punc_table).upper()
    
    re.sub(r'')
    if tmp_s == '':
        tmp_s = '[untitled]' 
    return tmp_s

In [90]:
变量 = '第一期 第九3批'

In [91]:
pattern = re.compile(r'[\(（\<《\[【\{「]第*[\u96f6\u3007\u4e00\u4e8c\u4e09\u56db\u4e94\u516d\u4e03\u516b\u4e5d\u5341\d]{1,5}[\u4e00-\u9fa5]*[\)）\>》\]】\}」]')
pattern = re.compile(r'第[\u96f6\u3007\u4e00\u4e8c\u4e09\u56db\u4e94\u516d\u4e03\u516b\u4e5d\u5341\d]{1,5}[次|批|期|号|卷]')
result = re.findall(pattern,变量)
for token in result:
    变量 = 变量.replace(token,token*10)

In [92]:
print(result)

第一期第一期第一期第一期第一期第一期第一期第一期第一期第一期 第九3批第九3批第九3批第九3批第九3批第九3批第九3批第九3批第九3批第九3批


In [ ]:
中国金融期货交易所关于发布国债期货合约TF2112、T2112和TS2112可交割国债的通知
Index:	11571
String:	中国金融期货交易所关于发布国债期货合约TF2203、T2203和TS2203可交割国债的通知

In [94]:
fuzz.ratio('中国金融期货交易所关于发布国债期货合约TF2112、T2112和TS2112可交割国债的通知', '中国金融期货交易所关于发布国债期货合约TF2203、T2203和TS2203可交割国债的通知')

80

In [ ]:
pdata4[pd.concat([(pdata3[0] == 1),(pdata3[0] == 1)],axis=0)]

In [ ]:
pdata4 = pd.concat([pdata3,pdata3],axis=0)
pdata4

In [ ]:
pdata3 = pd.DataFrame([[1,1],[2,2],[1,1],[1,1],[1,''],[1,'']])
pdata3

In [ ]:
pdata3[1].duplicated()

In [ ]:
not_dup_index = pdata3[1].duplicated() == False
not_dup_index

In [ ]:
empty_index = pdata3[1] == ''
empty_index

In [ ]:
pdata3[not_dup_index|empty_index]

In [ ]:
not_dup_index = pdata3[1].duplicated() == False
empty_index = pdata3[1] == ''
pdata3[not_dup_index|empty_index]

In [ ]:
pdata3.duplicated(keep=False)

In [ ]:
pdata3.drop_duplicates(subset=0).

In [ ]:
pdata3.duplicated(keep=False)

In [ ]:
pdata.head()

In [ ]:
%%time
pdata2 = pdata.drop_duplicates(subset='identifier_standard')

In [ ]:
pdata[pdata.duplicated(subset='identifier_standard')].id

In [ ]:
(pdata2.id == '207200692096').sum()

In [ ]:
(pdata.id == '207200692096').sum()

In [ ]:
len(pdata2)

In [ ]:
len(pdata)

In [ ]:
df = pdata[pdata.duplicated(subset='identifier_standard',keep=False)]
df

In [ ]:
df.groupby(df.identifier_standard)

In [ ]:
{j[0]:j[1].id.tolist()}

In [ ]:
import json

In [ ]:
import redis

In [ ]:
pdata = h5file.pdata

In [ ]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [ ]:
def taskDispatch(type_id, pdata):
    """
    1. 先hash再辅助判断
    2. doi先去重再hash
    3. 直接hash
    """
    if type_id in [1, 4, 8, 9, 12, 14]:
        hash_and_judge(h5file)
    elif type_id in [3, 5, 6, 7, 11]:
        dedup_and_hashe(h5file)
    # others 2 10 13 0
    else:
        hash_only(h5file)

In [53]:
for i in r.keys('2021*'):
    print (i)
    break

b'20212310947'


In [74]:
len(r.keys('[0-9]'*19))

4710943

In [75]:
fuzz.ratio('', '')

100

In [80]:
pd.read_hdf(path_or_buf='./data/4_2021.h5', start=1, stop=100)

,id,title,language,creator,creator_institution,source_file
1,20468132472,我国生鲜乳价格风险测度与管理研究,ZH,刘秉华,河北农业大学,111
2,20468466751,冀东地区土壤重金属元素基线值确定,ZH,焦建,河北地质大学,111
3,20468563375,氧化石墨烯的飞秒激光结构化处理及其浸润性调控,ZH,王飞跃,中国科学院大学(中国科学院长春光学精密机械与物理研究所),111
4,20468594821,铝合金的机器人气囊柔性抛光关键技术研究,ZH,邱磊,厦门理工学院,111
5,20468595217,智慧校园APP交互界面设计实践与研究 ——以“We辽师”为例,ZH,薛天,辽宁师范大学,111
...,...,...,...,...,...,...
5,20468594579,建安士风与公宴诗赋,ZH,蒋聪慧,辽宁师范大学,391
6,20468595190,北方旱作春玉米水分生产力时空变化的模拟研究,ZH,李威,中国农业科学院,391
0,20467257089,Improving student reading achievement through ...,EN,"Johnson, Shawn M.",University of Delaware,772
1,20467372234,CYRI-A is recruited to macropinocytic cups and...,EN,"Le, Hoang Anh",University of Glasgow,772


In [ ]:
import time

In [ ]:
for i, j in dic1.items():
    print('index:',i)
    print('dupl:',j)
    time.sleep(1)

In [ ]:
len(pdata)

In [ ]:
pdata.id[pdata['identifier_pisbn'] == '9787520172578'].tolist()

In [ ]:
pdata['id'][pdata['identifier_pisbn'] == '9787520172578'].tolist()

In [ ]:
pdata.loc[pdata['identifier_pisbn'] == '9787520172578']

In [ ]:
i_p_count = pdata['identifier_pisbn'].value_counts()
i_p_count

In [ ]:
i_p_count[1:]

In [ ]:
for i,j in i_p_count[i_p_count > 1].iteritems():
    print(i,j)
    time.sleep(1)

In [ ]:
import time

In [ ]:
i_p_count[i_p_count > 1]

In [ ]:
(pdata.language == 'ZH').count()

In [ ]:
(pdata.language == 'ZH').sum()

In [ ]:
pdata.loc[pdata.language == 'ZH', 'language']

In [ ]:
pdata.language.unique()

In [ ]:
pdata1 = pd.read_hdf('data/3_2021.h5')
pdata1.head()

In [ ]:
len(pdata1)

In [ ]:
print(len(pdata1.identifier_doi.unique()))

In [ ]:
pdata1.identifier_doi.unique()

In [ ]:
pdata1[pdata1.identifier_doi == '10.1016/j.scitotenv.2020.141974']

In [ ]:
(pdata1[pdata1.language == 'EN'].identifier_doi.value_counts()>2).sum()

In [ ]:
pdata1['identifier_doi'][100:130]

In [ ]:
len(pdata1.identifier_doi.unique())